### degrade_image_shift

In [3]:
import cv2
import numpy as np
import os

In [4]:
# Папка с оригинальными изображениями
input_folder = "./content/DIV2K"



### filtered and  shifted

In [ ]:
output_folder = "./content/filtered_shifted" 
os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения через смещение + снижение разрешения 
def degrade_image_shift(img, scale=5, shift_range=5):
    h, w = img.shape[:2]

    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

    # 2 Генерация случайного сдвига
    shift_x = np.random.uniform(-shift_range, shift_range)  # Горизонтальный сдвиг
    shift_y = np.random.uniform(-shift_range, shift_range)  # Вертикальный сдвиг

    # 3 Матрица сдвига
    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # 4 Применяем сдвиг
    shifted = cv2.warpAffine(blurred, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # # 5 Уменьшаем разрешение (имитация ухудшения)
    # small = cv2.resize(shifted, (w // scale, h // scale), interpolation=cv2.INTER_LINEAR)

    # # 6 Увеличиваем обратно
    # degraded = cv2.resize(small, (w, h), interpolation=cv2.INTER_LINEAR)

    # return degraded
    return shifted

# Количество ухудшенных версий для каждого изображения
num_versions = 10  

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png", ".jpg"))]

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # Генерируем несколько ухудшенных версий
    for i in range(num_versions):
        degraded_img = degrade_image_shift(img)

        # Сохраняем ухудшенное изображение
        save_path = f"{output_folder}/{img_name[:-4]}_v{i}.jpg"
        cv2.imwrite(save_path, degraded_img)

print("Good")

Good


### filtered, shifted and rotated

In [ ]:
output_folder = "./content/filtered_shifted_rotated_BORDER_REPLICATE"
os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения 
def degrade_image_transform(img, scale=5, shift_range=5, rotation_range=10):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)  # Центр изображения

    # 1 Фильтрация (усредняющее размытие)
    blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

    # 2 Генерируем случайные параметры
    shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
    shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
    angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах

    # 3 Генерируем координаты для выходного изображения
    x_out, y_out = np.meshgrid(np.arange(w), np.arange(h))  

    # 4 Применяем преобразование подобия 
    angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
    cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)

    x_in =  (cos_fi * (x_out - w // 2) + sin_fi * (y_out - h // 2)) + center[0] + shift_x
    y_in =  (-sin_fi * (x_out - w // 2) + cos_fi * (y_out - h // 2)) + center[1] + shift_y


    # 5 Используем `cv2.remap()` для выборки пикселей с интерполяцией
    map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)# cv2.remap() требует, чтобы координаты были в формате float32, иначе OpenCV вызовет ошибку
    # transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    # transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)


    return transformed

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png", ".jpg"))]

# Количество ухудшенных версий для каждого изображения
num_versions = 10 

for img_name in image_files:
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # Генерируем несколько ухудшенных версий
    for i in range(num_versions):
        degraded_img = degrade_image_transform(img)

        # Сохраняем ухудшенное изображение
        save_path = f"{output_folder}/{img_name[:-4]}_v{i}.jpg"
        cv2.imwrite(save_path, degraded_img)


print("Good")

Good


### filtered, shifted, rotated and scaled

In [17]:
output_folder = "./content/filtered_shifted_rotated_scaled"
os.makedirs(output_folder, exist_ok=True)  # Создаем папку для сохранения

# Функция ухудшения через преобразование подобия
def degrade_image_transform(img, scale=5, shift_range=5, rotation_range=10):
	h, w = img.shape[:2]
	center = (w // 2, h // 2)  # Центр изображения

	# 1 Фильтрация (усредняющее размытие)
	blurred = cv2.blur(img, (5, 5), borderType=cv2.BORDER_REPLICATE)

	# 2 Генерируем случайные параметры
	shift_x = np.random.uniform(-shift_range, shift_range)  # Сдвиг по X
	shift_y = np.random.uniform(-shift_range, shift_range)  # Сдвиг по Y
	angle = np.random.uniform(-rotation_range, rotation_range)  # Поворот в градусах
	k = np.random.uniform(scale - 0.5, scale + 0.5)  # Коэффициент уменьшения

	# 3 Генерируем координаты для выходного изображения
	new_h, new_w = int(h // k), int(w // k)  # Размер уменьшенного изображения
	x_out, y_out = np.meshgrid(np.arange(new_w), np.arange(new_h))  # Координаты в новом пространстве

	# 4 Применяем преобразование подобия 
	angle_rad = np.deg2rad(angle)  # Переводим градусы в радианы
	cos_fi, sin_fi = np.cos(angle_rad), np.sin(angle_rad)

	x_in = k * (cos_fi * (x_out - new_w // 2) + sin_fi * (y_out - new_h // 2)) + center[0] + shift_x
	y_in = k * (-sin_fi * (x_out - new_w // 2) + cos_fi * (y_out - new_h // 2)) + center[1] + shift_y

	# 5 Используем `cv2.remap()` для выборки пикселей с интерполяцией
	map_x, map_y = x_in.astype(np.float32), y_in.astype(np.float32)# cv2.remap() требует, чтобы координаты были в формате float32, иначе OpenCV вызовет ошибку
	transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
	# transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
	# transformed = cv2.remap(blurred, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

	# 6 Увеличиваем обратно
	degraded = cv2.resize(transformed, (w, h), interpolation=cv2.INTER_LINEAR)

	return degraded

# Обрабатываем изображения
image_files = [f for f in os.listdir(input_folder) if f.endswith((".png", ".jpg"))]

# Количество ухудшенных версий для каждого изображения
num_versions = 10 

for img_name in image_files:
	img_path = os.path.join(input_folder, img_name)
	img = cv2.imread(img_path)

	# Генерируем несколько ухудшенных версий
	for i in range(num_versions):
		degraded_img = degrade_image_transform(img)

		# Сохраняем ухудшенное изображение
		save_path = f"{output_folder}/{img_name[:-4]}_v{i}.jpg"
		cv2.imwrite(save_path, degraded_img)


print("Good")

Good
